<a href="https://colab.research.google.com/github/daver987/unsloth-mojo/blob/main/Unsloth_Mojo_Puzzles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unsloth `nf4` to `fp16/bf16` Challenge Mapped to Mojo

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
# Helpful functions used through the entire notebook
import torch
import torch.nn as nn
from transformers import set_seed
import time
import inspect
import os
major_version, minor_version = torch.cuda.get_device_capability()
HAS_BFLOAT16 = (major_version >= 8)
from inspect import currentframe as _C, getframeinfo
_F = lambda c: getframeinfo(c).lineno # Gets line number
WARN = lambda x: print(f"\033[31m{x}\033[0m") # Red colored warnings

# https://stackoverflow.com/questions/18425225/getting-the-name-of-a-variable-as-a-string
def NAME(var):
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    names = [var_name for var_name, var_val in callers_local_vars if var_val is var]
    return names[0] if len(names) != 0 else ""

def assert_same(x, y, line, dtype):
    assert(x.dtype == dtype)
    try: torch.testing.assert_close(x, y, check_stride = True)
    except Exception as error:
        raise RuntimeError(
            f"Failed allclose at line [{line}]: {NAME(x)}, {NAME(y)}\n{str(error)}"
        )

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-4c8a1b8c-de69-7d1e-2b39-17ce88820afa)


# Convert `nf4` to Triton. (Original Challenge)

<a name="NF4"></a>



1. Goal: Convert a `nf4` quantized tensor into `fp16` or `bf16` into a *single* Triton kernel The double dequant of the `absmax` and weight forming must be done in 1 Triton kernel. Must work on Tesla T4.
2. Must be faster than Unsloth's `fast_dequantize` by 1.15x or more, and not use large intermediate memory buffers.
3. Must not use `torch.compile`, but can use `trace.enabled` to help on writing Triton kernels.
4. Good material: [Unsloth `fast_dequantize` function](https://github.com/unslothai/unsloth/blob/main/unsloth/kernels/utils.py#L128), also [bitsandbytes `dequantize_blockwise`](https://github.com/bitsandbytes-foundation/bitsandbytes/blob/86b6c37a8ad448230cedb60753f63150b603a112/bitsandbytes/functional.py#L958)
5. Use `test_dequantize_function` to test your implementation.
6. No CUDA allowed. Custom CUDA inside of the Triton is allowed.
7. Watch Tim's videos on Youtube: [8-bit Optimizers](https://www.youtube.com/watch?v=2ETNONas068)

In [4]:
from bitsandbytes.nn import Linear4bit
from transformers.activations import ACT2FN
from unsloth.kernels.utils import fast_dequantize
from peft.utils.integrations import dequantize_module_weight as peft_dequantize

def unsloth_dequantize(weight):
    return fast_dequantize(weight.weight, weight.weight.quant_state)

def bnb_Linear4bit(hd, m, dtype = torch.float16):
    return Linear4bit(
        hd, m, bias = None,
        compute_dtype       = dtype,
        compress_statistics = True,
        quant_type          = "nf4",
    )

# [NEW] as at 18th Feb 2025
def assert_correct_bnb(weight, dtype):
    assert(weight.weight.dtype == torch.uint8)
    assert(weight.weight.quant_state.dtype == dtype)
    assert(weight.weight.quant_state.absmax.dtype == torch.uint8)
    assert(weight.weight.quant_state.code.dtype == torch.float32)
    assert(weight.weight.quant_state.offset.dtype == torch.float32)
    assert(weight.weight.quant_state.blocksize == 64)
    assert(weight.weight.quant_state.state2.absmax.dtype == torch.float32)
    assert(weight.weight.quant_state.state2.code.dtype == torch.float32)
    assert(weight.weight.quant_state.state2.blocksize == 256)

class MLP(nn.Module):
    def __init__(self, hd = 4096, m = 14336, dtype = torch.float16):
        super().__init__()
        self.gate_proj = bnb_Linear4bit(hd, m, dtype = dtype).to("cuda")
        self.up_proj   = bnb_Linear4bit(hd, m, dtype = dtype).to("cuda")
        self.down_proj = bnb_Linear4bit(m, hd, dtype = dtype).to("cuda")
        # [NEW] as at 18th Feb 2025
        self.gate_proj.weight.quant_state.dtype = dtype
        self.up_proj  .weight.quant_state.dtype = dtype
        self.down_proj.weight.quant_state.dtype = dtype
        self.act_fn = ACT2FN["silu"]
    def forward(self, x):
        return self.down_proj(self.act_fn(self.gate_proj(x)) * self.up_proj(x))

def mlp_forward(X, mlp, fx):
    up   = X @ fx(mlp.  up_proj).t()
    gate = X @ fx(mlp.gate_proj).t()
    h = mlp.act_fn(gate) * up
    down = h @ fx(mlp.down_proj).t()
    return down

def mlp_dequantize(X, mlp, fx):
    a = fx(mlp.  up_proj).t(); torch.cuda.synchronize()
    b = fx(mlp.gate_proj).t(); torch.cuda.synchronize()
    c = fx(mlp.down_proj).t(); torch.cuda.synchronize()
    return a, b, c

def test_dequantize(dequantize_fx):
    elapsed = 0
    options = [
        (2, 3333, 2048,  8192, 3407, torch.float16),
        (5,  777, 1024,  4096, 3409, torch.bfloat16),
        (3, 2048, 4096, 14336, 3408, torch.bfloat16),
    ]
    for (bsz, qlen, hd, m, seed, dt) in options:
        set_seed(seed)
        torch.set_default_dtype(torch.float32)
        mlp = MLP(hd = hd, m = m, dtype = dt)
        X = torch.randn((bsz, qlen, hd), device = "cuda", dtype = dt)
        torch.cuda.synchronize()

        # Warmup
        for _ in range(2):
            assert_same( mlp_forward(X, mlp, dequantize_fx), mlp(X), _F(_C()), dt)
            # [NEW] as at 18th Feb 2025
            assert_correct_bnb(mlp.  up_proj, dt)
            assert_correct_bnb(mlp.gate_proj, dt)
            assert_correct_bnb(mlp.down_proj, dt)
            a, b, c = mlp_dequantize(X, mlp, dequantize_fx)
            A, B, C = mlp_dequantize(X, mlp, unsloth_dequantize)
            assert_same(a, A, _F(_C()), dt)
            assert_same(b, B, _F(_C()), dt)
            assert_same(c, C, _F(_C()), dt)

        # Benchmarking
        torch.cuda.synchronize()
        start = time.time()
        for _ in range(1000): mlp_dequantize(X, mlp, dequantize_fx)
        elapsed += time.time() - start
    return elapsed

/tmp/ipython-input-1294683329.py:3: UserWarning: WARNING: Unsloth should be imported before [transformers] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth.kernels.utils import fast_dequantize


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


For example, we can test our implementation via:

In [5]:
from unsloth.kernels.utils import fast_dequantize

def unsloth_dequantize(weight):
    return fast_dequantize(weight.weight, weight.weight.quant_state)

test_dequantize(unsloth_dequantize)

3.713529586791992

The elapsed time for our implementation over 1000 trials is 5.38 seconds or so.

PEFT also has one, which should be mostly identical to Unsloth's version, albeit slightly slower.

In [6]:
from peft.utils.integrations import dequantize_module_weight as peft_dequantize

test_dequantize(peft_dequantize)

4.664241313934326

# Unsloth Harness

Write your Triton kernel below, and test it:

In [7]:
# from triton import jit
# import triton
# import triton.language as tl

# @triton.jit
# def _your_dequantize_nf4_kernel():
#     ### TRITON CODE GOES HERE
#     return

# def _your_dequantize_nf4(weight, quant_state):
#     ### SETUP TRITON LAUNCH HERE
#     return None

# def your_dequantize_nf4(weight):
#     return _your_dequantize_nf4(weight.weight.data, weight.weight.quant_state)

# NF4 To FP16/BF16 Dequant in Mojo

This notebook implements **bitsandbytes/Unsloth NF4 dequantization** as **pure Mojo GPU kernels**.

The Goal:
- Have a single fused kernel
- Convert an NF4 quantized tensor to FP16/BF16
- No large intermediate buffers
- No `torch.compile` or Max graph magic in the case of Mojo
- Must run on a T4
- No C++ or CUDA
- Do the full NF4 path including the “double dequant” scale reconstruction
- Correctness matched down to the small semantics details.

## What this kernel does

Given packed NF4 bytes (`uint8`) and the bnb quant state (`absmax_q`, `state2.code`, `state2.absmax`, `offset`), the kernel:

* Decodes NF4 codes using the standard 16-value codebook
* Reconstructs the per-block scale via `scale = state2.code[absmax_q] * state2.absmax[group] + offset`
* Multiplies, casts, and stores

### Exact bnb semantics

This version matches bnb behavior exactly, including:

* `blocksize = 64`, `state2.blocksize = 256`
* **high nibble = first weight**, low nibble = second
* separate **FP16** and **BF16** specialized kernels (no runtime dtype branching)

## Benchmarking

- Harness uses the same three `(hd, m)` configs as Unsloth’s `test_dequantize`
- Dequantizes the same three matrices per config (`up`, `gate`, `down`).
- Includes **mixed precision** (1×FP16 + 2×BF16) to mirror the original notebook’s dtype schedule

# Final Results Across Different GPUs

All runs were done at the same time on the same runtime

Totals for the dequant workload on each GPU.

| GPU | Unsloth `fast_dequantize` | GB/s  | Mojo kernel | GB/s  | Speedup |
| --- | --- | --- | --- | --- | --- |
| T4 | 3.6984355449676514 | 162.6315 | 3.461897315 | 173.7435 | 1.068x |
| L4 | 3.0031237602233887 | 200.2855 | 2.404278076 | 250.17166 | 1.249x |
| A100 - 40Gb | 1.2074337005615234 | 498.1492 | 0.656471045 | 916.2357 | 1.839x |
| H100 - PCIe | 0.6175928115844727 |  973.9139 | 0.408032219 | 1474.1047 | 1.514x |

# Install Mojo

In [8]:
!pip install -U -q mojo --index-url https://dl.modular.com/public/nightly/python/simple/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.1/22.1 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.3/101.3 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 591.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.3 MB/s eta 0:00:00


### Check that Mojo is available and import notebook

In [9]:
import mojo.notebook

### Check that we are on the GPU

In [10]:
%%mojo

from gpu.host import DeviceContext

fn kernel():
    print("Hello from the GPU")

def main():
    with DeviceContext() as ctx:
        ctx.enqueue_function_checked[kernel, kernel](grid_dim=1, block_dim=1)
        ctx.synchronize()

Hello from the GPU



# Mojo NF4 To F16/BF16 Kernels

In [11]:
%%mojo
from gpu import block_idx, thread_idx, barrier
from gpu.host import DeviceContext
from gpu.memory import AddressSpace
from memory import UnsafePointer, stack_allocation, bitcast
from math import fma, ceildiv

#######################
# Types and Constants #
#######################

comptime packed_dtype = DType.uint8
comptime f32_dtype    = DType.float32
comptime f16_dtype    = DType.float16
comptime bf16_dtype   = DType.bfloat16
comptime u32_dtype    = DType.uint32
comptime u16_dtype    = DType.uint16

comptime U8   = Scalar[packed_dtype]
comptime F32  = Scalar[f32_dtype]
comptime F16  = Scalar[f16_dtype]
comptime BF16 = Scalar[bf16_dtype]
comptime U32  = Scalar[u32_dtype]
comptime U16  = Scalar[u16_dtype]

# NF4 codebook (bitsandbytes)
comptime NF4_TABLE = InlineArray[F32, 16](
    -1.0,
    -0.6961928009986877,
    -0.5250730514526367,
    -0.39491748809814453,
    -0.28444138169288635,
    -0.18477343022823334,
    -0.09105003625154495,
    0.0,
    0.07958029955625534,
    0.16093020141124725,
    0.24611230194568634,
    0.33791524171829224,
    0.44070982933044434,
    0.5626170039176941,
    0.7229568362236023,
    1.0
)

# constants (bnb NF4)
comptime NF4_WEIGHTS_PER_BLOCK = 64   # 64 weights per NF4 block
comptime NF4_BYTES_PER_BLOCK   = 32   # 64 / 2 two 4bit weights per byte
comptime NF4_BLOCK_SHIFT       = 5    # log2(32) block_id = global_byte >> 5
comptime STATE2_BLOCKS         = 256  # 256 NF4 blocks per state2 group
comptime STATE2_SHIFT          = 8    # log2(256) group_id = block_id >> 8
comptime CODE2_SIZE            = 256  # state2.code lookup table size

# block config
comptime TILE_BYTES_X = 256  #  bytes processed per row tile (x)
comptime TILE_ROWS    = 4    #  rows processed per block     (y)
comptime THREADS_X    = 128  #  threads along (x)
comptime THREADS_Y    = 4    #  threads along (y)

# Each thread handles 1 byte at   `byte = block_y * TILE_BYTES_X + tx`
# plus an unrolled second byte at `byte + THREADS_X` -> 2 bytes / thread.

# So per row:
# 128 threads * 2 bytes/thread = 256 bytes

#############################
# Helpers: pack 2x16 -> u32 #
#############################

@always_inline
fn pack2_u16_to_u32(lo16: U16, hi16: U16) -> U32:
    """Packs two 16-bit unsigned integers into a single 32-bit unsigned integer.

    The low 16 bits contain the first value, and the high 16 bits contain the second value.

    Parameters:
        lo16: The lower 16-bit value
        hi16: The higher 16-bit value

    Returns:
        A 32-bit unsigned integer with the packed values.
    """
    return U32(lo16) | (U32(hi16) << 16)

@always_inline
fn pack2_f16_to_u32(a: F16, b: F16) -> U32:
    """Packs two 16-bit floating-point numbers (f16) into a single 32-bit unsigned integer.

    Parameters:
        a: The first f16 value
        b: The second f16 value

    Returns:
        A 32-bit unsigned integer with the packed f16 values.
    """
    var a_u16: U16 = bitcast[u16_dtype](a)
    var b_u16: U16 = bitcast[u16_dtype](b)
    return pack2_u16_to_u32(a_u16, b_u16)

@always_inline
fn pack2_bf16_to_u32(a: BF16, b: BF16) -> U32:
    """Packs two bfloat16 values into a single 32-bit unsigned integer.

    Parameters:
        a: The first bf16 value
        b: The second bf16 value

    Returns:
        A 32-bit unsigned integer with the packed bf16 values
    """
    var a_u16: U16 = bitcast[u16_dtype](a)
    var b_u16: U16 = bitcast[u16_dtype](b)
    return pack2_u16_to_u32(a_u16, b_u16)


#####################################
# GPU kernel: NF4 -> packed u32 (F16)#
#####################################
fn nf4_dequant_bnb_unroll2_packed_u32_f16(
    packed_ptr:    UnsafePointer[U8,  MutAnyOrigin],  # W packed (nf4), length = n_rows * n_bytes_per_row
    absmax_q_ptr:  UnsafePointer[U8,  MutAnyOrigin],  # quant_state.absmax (uint8), length = n_nf4_blocks
    code2_ptr:     UnsafePointer[F32, MutAnyOrigin],  # quant_state.state2.code (float32), length = 256
    absmax2_ptr:   UnsafePointer[F32, MutAnyOrigin],  # quant_state.state2.absmax (float32), length = n_groups
    offset:        F32,                               # quant_state.offset
    out_ptr:       UnsafePointer[U32, MutAnyOrigin],  # output: u32 per packed byte (2x f16)
    n_rows:        Int,
    n_bytes_per_row: Int,
):
    """Dequantizes NF4 packed weights to packed u32 (containing two f16 values) on GPU with unrolling.

    This kernel processes NF4 quantized weights, dequantizes them using the provided quantization states
    Stores the results as packed u32 values (each holding two f16 floats).
    Uses shared memory for the NF4 table and processes tiles of data with unrolling for efficiency.

    Parameters:
        packed_ptr: Pointer to packed NF4 weights (uint8)
        absmax_q_ptr: Pointer to quantized absmax values (uint8)
        code2_ptr: Pointer to state2 code table (float32)
        absmax2_ptr: Pointer to state2 absmax values (float32)
        offset: Quantization offset (float32)
        out_ptr: Pointer to output packed u32 (each u32 holds two f16)
        n_rows: Number of rows in the weight matrix.
        n_bytes_per_row: Number of packed bytes per row

    Note: Uses unrolling to process two bytes per thread for better performance.
    """
    # Shared NF4 table
    var sh_nf4 = stack_allocation[16, F32, address_space = AddressSpace.SHARED]()
    var tx = Int(thread_idx.x)
    var ty = Int(thread_idx.y)

    if ty == 0 and tx < 16:
        sh_nf4[tx] = NF4_TABLE[tx]
    barrier()

    var row  = Int(block_idx.x) * TILE_ROWS + ty
    var byte = Int(block_idx.y) * TILE_BYTES_X + tx
    if row >= n_rows or byte >= n_bytes_per_row:
        return

    var row_base = row * n_bytes_per_row
    var gbyte0   = row_base + byte

    ############################
    # Byte 0 (bnb nibble order)#
    ############################
    var p0_u8: U8 = packed_ptr[gbyte0]
    var p0: Int = Int(p0_u8)

    # bnb semantics:
    # first weight  = HIGH nibble
    # second weight = LOW  nibble
    var idx0_first:  Int = (p0 >> 4) & 0x0F
    var idx0_second: Int =  p0       & 0x0F

    var w0n: F32 = sh_nf4[idx0_first]
    var w1n: F32 = sh_nf4[idx0_second]

    var block0: Int = gbyte0 >> NF4_BLOCK_SHIFT
    var q0: U8 = absmax_q_ptr[block0]
    var code0: F32 = code2_ptr[Int(q0)]
    var absmax20: F32 = absmax2_ptr[block0 >> STATE2_SHIFT]
    var scale0: F32 = fma(code0, absmax20, offset)

    var w0: F32 = w0n * scale0
    var w1: F32 = w1n * scale0

    var h0: F16 = F16(w0)
    var h1: F16 = F16(w1)
    out_ptr[gbyte0] = pack2_f16_to_u32(h0, h1)

    ###############################
    # Byte 1 (unroll +128 bytes)  #
    ###############################
    var byte1 = byte + THREADS_X
    if byte1 < n_bytes_per_row:
        var gbyte1 = gbyte0 + THREADS_X

        var p1_u8: U8 = packed_ptr[gbyte1]
        var p1: Int = Int(p1_u8)

        var idx1_first:  Int = (p1 >> 4) & 0x0F
        var idx1_second: Int =  p1       & 0x0F

        var w2n: F32 = sh_nf4[idx1_first]
        var w3n: F32 = sh_nf4[idx1_second]

        # +128 bytes = +4 NF4 blocks (32 bytes/block)
        var block1: Int = block0 + 4
        var q1: U8 = absmax_q_ptr[block1]
        var code1: F32 = code2_ptr[Int(q1)]
        var absmax21: F32 = absmax2_ptr[block1 >> STATE2_SHIFT]
        var scale1: F32 = fma(code1, absmax21, offset)

        var w2: F32 = w2n * scale1
        var w3: F32 = w3n * scale1

        var h2: F16 = F16(w2)
        var h3: F16 = F16(w3)
        out_ptr[gbyte1] = pack2_f16_to_u32(h2, h3)

########################################
# GPU kernel: NF4 -> packed u32 (BF16) #
########################################
fn nf4_dequant_bnb_unroll2_packed_u32_bf16(
    packed_ptr:    UnsafePointer[U8,  MutAnyOrigin],
    absmax_q_ptr:  UnsafePointer[U8,  MutAnyOrigin],
    code2_ptr:     UnsafePointer[F32, MutAnyOrigin],
    absmax2_ptr:   UnsafePointer[F32, MutAnyOrigin],
    offset:        F32,
    out_ptr:       UnsafePointer[U32, MutAnyOrigin],
    n_rows:        Int,
    n_bytes_per_row: Int,
):
    """Dequantizes NF4 packed weights to packed u32 (containing two bf16 values) on GPU with unrolling.

    This kernel processes NF4 quantized weights, dequantizes them using the provided quantization states
    Stores the results as packed u32 values (each holding two bf16 floats).
    Uses shared memory for the NF4 table and processes tiles of data with unrolling for efficiency.

    Parameters:
        packed_ptr: Pointer to packed NF4 weights (uint8)
        absmax_q_ptr: Pointer to quantized absmax values (uint8)
        code2_ptr: Pointer to state2 code table (float32)
        absmax2_ptr: Pointer to state2 absmax values (float32)
        offset: Quantization offset (float32)
        out_ptr: Pointer to output packed u32 (each u32 holds two bf16)
        n_rows: Number of rows in the weight matrix.
        n_bytes_per_row: Number of packed bytes per row.

    Note: Uses unrolling to process two bytes per thread for better performance.
    """
    var sh_nf4 = stack_allocation[16, F32, address_space = AddressSpace.SHARED]()
    var tx = Int(thread_idx.x)
    var ty = Int(thread_idx.y)

    if ty == 0 and tx < 16:
        sh_nf4[tx] = NF4_TABLE[tx]
    barrier()

    var row  = Int(block_idx.x) * TILE_ROWS + ty
    var byte = Int(block_idx.y) * TILE_BYTES_X + tx
    if row >= n_rows or byte >= n_bytes_per_row:
        return

    var row_base = row * n_bytes_per_row
    var gbyte0   = row_base + byte

    var p0_u8: U8 = packed_ptr[gbyte0]
    var p0: Int = Int(p0_u8)

    var idx0_first:  Int = (p0 >> 4) & 0x0F
    var idx0_second: Int =  p0       & 0x0F

    var w0n: F32 = sh_nf4[idx0_first]
    var w1n: F32 = sh_nf4[idx0_second]

    var block0: Int = gbyte0 >> NF4_BLOCK_SHIFT
    var q0: U8 = absmax_q_ptr[block0]
    var code0: F32 = code2_ptr[Int(q0)]
    var absmax20: F32 = absmax2_ptr[block0 >> STATE2_SHIFT]
    var scale0: F32 = fma(code0, absmax20, offset)

    var w0: F32 = w0n * scale0
    var w1: F32 = w1n * scale0

    var b0: BF16 = BF16(w0)
    var b1: BF16 = BF16(w1)
    out_ptr[gbyte0] = pack2_bf16_to_u32(b0, b1)

    var byte1 = byte + THREADS_X
    if byte1 < n_bytes_per_row:
        var gbyte1 = gbyte0 + THREADS_X

        var p1_u8: U8 = packed_ptr[gbyte1]
        var p1: Int = Int(p1_u8)

        var idx1_first:  Int = (p1 >> 4) & 0x0F
        var idx1_second: Int =  p1       & 0x0F

        var w2n: F32 = sh_nf4[idx1_first]
        var w3n: F32 = sh_nf4[idx1_second]

        var block1: Int = block0 + 4
        var q1: U8 = absmax_q_ptr[block1]
        var code1: F32 = code2_ptr[Int(q1)]
        var absmax21: F32 = absmax2_ptr[block1 >> STATE2_SHIFT]
        var scale1: F32 = fma(code1, absmax21, offset)

        var w2: F32 = w2n * scale1
        var w3: F32 = w3n * scale1

        var b2: BF16 = BF16(w2)
        var b3: BF16 = BF16(w3)
        out_ptr[gbyte1] = pack2_bf16_to_u32(b2, b3)

#################
# CPU reference #
#################
fn nf4_dequant_cpu_bnb_f16(
    packed:   UnsafePointer[U8,  MutAnyOrigin],
    absmax_q: UnsafePointer[U8,  MutAnyOrigin],
    code2:    UnsafePointer[F32, MutAnyOrigin],
    absmax2:  UnsafePointer[F32, MutAnyOrigin],
    offset:   F32,
    output:   UnsafePointer[F16, MutAnyOrigin],
    n_rows:   Int,
    n_cols:   Int,
):
    """CPU reference implementation for dequantizing NF4 weights to f16.

    This function dequantizes NF4 packed weights using the bitsandbytes (bnb) semantics
    and stores the results as f16 values.

    Parameters:
        packed: Pointer to packed NF4 weights (uint8)
        absmax_q: Pointer to quantized absmax values (uint8)
        code2: Pointer to state2 code table (float32)
        absmax2: Pointer to state2 absmax values (float32)
        offset: Quantization offset (float32)
        output: Pointer to output f16 weights
        n_rows: Number of rows in the matrix
        n_cols: Number of columns in the matrix
    """
    var n_bytes_per_row = n_cols >> 1
    for row in range(n_rows):
        var row_byte_base = row * n_bytes_per_row
        var row_out_base  = row * n_cols
        for byte_idx in range(n_bytes_per_row):
            var gbyte = row_byte_base + byte_idx
            var pb: U8 = packed[gbyte]
            var p: Int = Int(pb)

            var idx_first:  Int = (p >> 4) & 0x0F
            var idx_second: Int =  p       & 0x0F

            var w0n: F32 = NF4_TABLE[idx_first]
            var w1n: F32 = NF4_TABLE[idx_second]

            var block: Int = gbyte >> NF4_BLOCK_SHIFT
            var q: U8 = absmax_q[block]
            var codev: F32 = code2[Int(q)]
            var absmax2v: F32 = absmax2[block >> STATE2_SHIFT]
            var scale: F32 = fma(codev, absmax2v, offset)

            var col0 = byte_idx << 1
            output[row_out_base + col0]     = F16(w0n * scale)
            output[row_out_base + col0 + 1] = F16(w1n * scale)

fn nf4_dequant_cpu_bnb_bf16(
    packed:   UnsafePointer[U8,  MutAnyOrigin],
    absmax_q: UnsafePointer[U8,  MutAnyOrigin],
    code2:    UnsafePointer[F32, MutAnyOrigin],
    absmax2:  UnsafePointer[F32, MutAnyOrigin],
    offset:   F32,
    output:   UnsafePointer[BF16, MutAnyOrigin],
    n_rows:   Int,
    n_cols:   Int,
):
    """CPU reference implementation for dequantizing NF4 weights to bf16.

    This function dequantizes NF4 packed weights using the bitsandbytes semantics
    and stores the results as bf16 values.

    Parameters:
        packed: Pointer to packed NF4 weights (uint8)
        absmax_q: Pointer to quantized absmax values (uint8)
        code2: Pointer to state2 code table (float32)
        absmax2: Pointer to state2 absmax values (float32)
        offset: Quantization offset (float32)
        output: Pointer to output bf16 weights
        n_rows: Number of rows in the matrix
        n_cols: Number of columns in the matrix
    """
    var n_bytes_per_row = n_cols >> 1
    for row in range(n_rows):
        var row_byte_base = row * n_bytes_per_row
        var row_out_base  = row * n_cols
        for byte_idx in range(n_bytes_per_row):
            var gbyte = row_byte_base + byte_idx
            var pb: U8 = packed[gbyte]
            var p: Int = Int(pb)

            var idx_first:  Int = (p >> 4) & 0x0F
            var idx_second: Int =  p       & 0x0F

            var w0n: F32 = NF4_TABLE[idx_first]
            var w1n: F32 = NF4_TABLE[idx_second]

            var block: Int = gbyte >> NF4_BLOCK_SHIFT
            var q: U8 = absmax_q[block]
            var codev: F32 = code2[Int(q)]
            var absmax2v: F32 = absmax2[block >> STATE2_SHIFT]
            var scale: F32 = fma(codev, absmax2v, offset)

            var col0 = byte_idx << 1
            output[row_out_base + col0]     = BF16(w0n * scale)
            output[row_out_base + col0 + 1] = BF16(w1n * scale)

#################################################
# Correctness tests (cross 256-block boundary)  #
#################################################
def run_correctness_test_f16(ctx: DeviceContext):
    """Runs correctness test for NF4 dequantization to f16.

    This test compares GPU dequantization results with CPU reference implementation
    for f16 output, checking for maximum absolute difference.

    Parameters:
        ctx: The device context for GPU operations.

    Note: Tests across 256-block boundaries to ensure correctness in grouping.
    """
    var n_rows: Int = 2
    var n_cols: Int = 16384
    var n_weights: Int = n_rows * n_cols
    var n_bytes: Int = n_weights >> 1
    var n_bytes_per_row: Int = n_cols >> 1

    var n_nf4_blocks: Int = ceildiv(n_bytes, NF4_BYTES_PER_BLOCK)
    var n_groups: Int = ceildiv(n_nf4_blocks, STATE2_BLOCKS)
    if n_groups < 1: n_groups = 1

    var offset: F32 = 0.01

    var packed_host   = ctx.enqueue_create_host_buffer[packed_dtype](n_bytes)
    var absmax_q_host = ctx.enqueue_create_host_buffer[packed_dtype](n_nf4_blocks)
    var code2_host    = ctx.enqueue_create_host_buffer[f32_dtype](CODE2_SIZE)
    var absmax2_host  = ctx.enqueue_create_host_buffer[f32_dtype](n_groups)
    var out_cpu_host  = ctx.enqueue_create_host_buffer[f16_dtype](n_weights)

    ctx.synchronize()

    # high nibble = first, low nibble = second
    for i in range(n_bytes):
        var first:  Int = (i + 1) & 15
        var second: Int = i & 15
        packed_host[i] = U8((first << 4) | second)

    for i in range(n_nf4_blocks):
        absmax_q_host[i] = U8(i & 255)

    for i in range(CODE2_SIZE):
        var x: Float64 = (Float64(i) - 127.5) / 127.5
        code2_host[i] = F32(x)

    for g in range(n_groups):
        absmax2_host[g] = F32(1.0 + Float64(g))

    nf4_dequant_cpu_bnb_f16(
        packed_host.unsafe_ptr(),
        absmax_q_host.unsafe_ptr(),
        code2_host.unsafe_ptr(),
        absmax2_host.unsafe_ptr(),
        offset,
        out_cpu_host.unsafe_ptr(),
        n_rows,
        n_cols,
    )

    var packed_dev   = ctx.enqueue_create_buffer[packed_dtype](n_bytes)
    var absmax_q_dev = ctx.enqueue_create_buffer[packed_dtype](n_nf4_blocks)
    var code2_dev    = ctx.enqueue_create_buffer[f32_dtype](CODE2_SIZE)
    var absmax2_dev  = ctx.enqueue_create_buffer[f32_dtype](n_groups)
    var out_dev      = ctx.enqueue_create_buffer[u32_dtype](n_bytes)

    ctx.enqueue_copy(dst_buf=packed_dev,   src_buf=packed_host)
    ctx.enqueue_copy(dst_buf=absmax_q_dev, src_buf=absmax_q_host)
    ctx.enqueue_copy(dst_buf=code2_dev,    src_buf=code2_host)
    ctx.enqueue_copy(dst_buf=absmax2_dev,  src_buf=absmax2_host)

    var grid_x: Int = ceildiv(n_rows, TILE_ROWS)
    var grid_y: Int = ceildiv(n_bytes_per_row, TILE_BYTES_X)

    var compiled = ctx.compile_function_checked[
        nf4_dequant_bnb_unroll2_packed_u32_f16, nf4_dequant_bnb_unroll2_packed_u32_f16
    ]()

    @always_inline
    @parameter
    fn run_kernel(test_ctx: DeviceContext) raises:
        test_ctx.enqueue_function_checked(
            compiled,
            packed_dev,
            absmax_q_dev,
            code2_dev,
            absmax2_dev,
            offset,
            out_dev,
            n_rows,
            n_bytes_per_row,
            grid_dim=(grid_x, grid_y),
            block_dim=(THREADS_X, THREADS_Y),
        )

    run_kernel(ctx)
    ctx.synchronize()

    with out_dev.map_to_host() as out_gpu_host:
        var max_abs_diff: F32 = 0.0
        for i in range(n_bytes):
            var w: U32 = out_gpu_host[i]
            var lo: U16 = U16(w & 0xFFFF)
            var hi: U16 = U16((w >> 16) & 0xFFFF)
            var g0: F16 = bitcast[f16_dtype](lo)
            var g1: F16 = bitcast[f16_dtype](hi)

            var idx0 = i << 1
            var idx1 = idx0 + 1

            var c0: F32 = F32(out_cpu_host[idx0])
            var c1: F32 = F32(out_cpu_host[idx1])
            var f0: F32 = F32(g0)
            var f1: F32 = F32(g1)

            var d0: F32 = c0 - f0
            var d1: F32 = c1 - f1
            if d0 < 0.0: d0 = -d0
            if d1 < 0.0: d1 = -d1
            if d0 > max_abs_diff: max_abs_diff = d0
            if d1 > max_abs_diff: max_abs_diff = d1

        print("[F16] max_abs_diff =", max_abs_diff)
        print("[F16] Correctness:", "PASSED" if max_abs_diff < 1e-05 else "FAILED")

def run_correctness_test_bf16(ctx: DeviceContext):
    """Runs correctness test for NF4 dequantization to bf16.

    This test compares GPU dequantization results with CPU reference implementation
    for bf16 output, checking for maximum absolute difference.

    Parameters:
        ctx: The device context for GPU operations

    Note: Tests across 256-block boundaries to ensure correctness in grouping.
    """
    var n_rows: Int = 2
    var n_cols: Int = 16384
    var n_weights: Int = n_rows * n_cols
    var n_bytes: Int = n_weights >> 1
    var n_bytes_per_row: Int = n_cols >> 1

    var n_nf4_blocks: Int = ceildiv(n_bytes, NF4_BYTES_PER_BLOCK)
    var n_groups: Int = ceildiv(n_nf4_blocks, STATE2_BLOCKS)
    if n_groups < 1: n_groups = 1

    var offset: F32 = 0.01

    var packed_host   = ctx.enqueue_create_host_buffer[packed_dtype](n_bytes)
    var absmax_q_host = ctx.enqueue_create_host_buffer[packed_dtype](n_nf4_blocks)
    var code2_host    = ctx.enqueue_create_host_buffer[f32_dtype](CODE2_SIZE)
    var absmax2_host  = ctx.enqueue_create_host_buffer[f32_dtype](n_groups)
    var out_cpu_host  = ctx.enqueue_create_host_buffer[bf16_dtype](n_weights)

    ctx.synchronize()

    for i in range(n_bytes):
        var first:  Int = (i + 1) & 15
        var second: Int = i & 15
        packed_host[i] = U8((first << 4) | second)

    for i in range(n_nf4_blocks):
        absmax_q_host[i] = U8(i & 255)

    for i in range(CODE2_SIZE):
        var x: Float64 = (Float64(i) - 127.5) / 127.5
        code2_host[i] = F32(x)

    for g in range(n_groups):
        absmax2_host[g] = F32(1.0 + Float64(g))

    nf4_dequant_cpu_bnb_bf16(
        packed_host.unsafe_ptr(),
        absmax_q_host.unsafe_ptr(),
        code2_host.unsafe_ptr(),
        absmax2_host.unsafe_ptr(),
        offset,
        out_cpu_host.unsafe_ptr(),
        n_rows,
        n_cols,
    )

    var packed_dev   = ctx.enqueue_create_buffer[packed_dtype](n_bytes)
    var absmax_q_dev = ctx.enqueue_create_buffer[packed_dtype](n_nf4_blocks)
    var code2_dev    = ctx.enqueue_create_buffer[f32_dtype](CODE2_SIZE)
    var absmax2_dev  = ctx.enqueue_create_buffer[f32_dtype](n_groups)
    var out_dev      = ctx.enqueue_create_buffer[u32_dtype](n_bytes)

    ctx.enqueue_copy(dst_buf=packed_dev,   src_buf=packed_host)
    ctx.enqueue_copy(dst_buf=absmax_q_dev, src_buf=absmax_q_host)
    ctx.enqueue_copy(dst_buf=code2_dev,    src_buf=code2_host)
    ctx.enqueue_copy(dst_buf=absmax2_dev,  src_buf=absmax2_host)

    var grid_x: Int = ceildiv(n_rows, TILE_ROWS)
    var grid_y: Int = ceildiv(n_bytes_per_row, TILE_BYTES_X)

    var compiled = ctx.compile_function_checked[
        nf4_dequant_bnb_unroll2_packed_u32_bf16, nf4_dequant_bnb_unroll2_packed_u32_bf16
    ]()

    @always_inline
    @parameter
    fn run_kernel(test_ctx: DeviceContext) raises:
        test_ctx.enqueue_function_checked(
            compiled,
            packed_dev,
            absmax_q_dev,
            code2_dev,
            absmax2_dev,
            offset,
            out_dev,
            n_rows,
            n_bytes_per_row,
            grid_dim=(grid_x, grid_y),
            block_dim=(THREADS_X, THREADS_Y),
        )

    run_kernel(ctx)
    ctx.synchronize()

    with out_dev.map_to_host() as out_gpu_host:
        var max_abs_diff: F32 = 0.0
        for i in range(n_bytes):
            var w: U32 = out_gpu_host[i]
            var lo: U16 = U16(w & 0xFFFF)
            var hi: U16 = U16((w >> 16) & 0xFFFF)
            var g0: BF16 = bitcast[bf16_dtype](lo)
            var g1: BF16 = bitcast[bf16_dtype](hi)

            var idx0 = i << 1
            var idx1 = idx0 + 1

            var c0: F32 = F32(out_cpu_host[idx0])
            var c1: F32 = F32(out_cpu_host[idx1])
            var f0: F32 = F32(g0)
            var f1: F32 = F32(g1)

            var d0: F32 = c0 - f0
            var d1: F32 = c1 - f1
            if d0 < 0.0: d0 = -d0
            if d1 < 0.0: d1 = -d1
            if d0 > max_abs_diff: max_abs_diff = d0
            if d1 > max_abs_diff: max_abs_diff = d1

        print("[BF16] max_abs_diff =", max_abs_diff)
        print("[BF16] Correctness:", "PASSED" if max_abs_diff < 1e-05 else "FAILED")

###################################
# Bench a single 2D weight matrix #
####################################
fn bench_layer_unroll2_f16(ctx: DeviceContext, n_rows: Int, n_cols: Int, num_iters: Int) raises -> Float64:
    """Benchmarks the NF4 dequantization kernel for f16 output on a single layer.

    Measures the execution time for dequantizing a 2D weight matrix to packed u32 (f16) over multiple iterations.

    Parameters:
        ctx: The device context for GPU operations
        n_rows: Number of rows in the matrix
        n_cols: Number of columns in the matrix
        num_iters: Number of iterations to run the benchmark

    Returns:
        Total execution time in seconds.

    Note: Initializes test data with patterns to simulate real quantization states.
    """
    var n_weights = n_rows * n_cols
    var n_bytes = n_weights >> 1
    var n_bytes_per_row = n_cols >> 1

    var n_nf4_blocks: Int = ceildiv(n_bytes, NF4_BYTES_PER_BLOCK)
    var n_groups: Int = ceildiv(n_nf4_blocks, STATE2_BLOCKS)
    if n_groups < 1: n_groups = 1

    var offset: F32 = 0.01

    var packed_host   = ctx.enqueue_create_host_buffer[packed_dtype](n_bytes)
    var absmax_q_host = ctx.enqueue_create_host_buffer[packed_dtype](n_nf4_blocks)
    var code2_host    = ctx.enqueue_create_host_buffer[f32_dtype](CODE2_SIZE)
    var absmax2_host  = ctx.enqueue_create_host_buffer[f32_dtype](n_groups)

    ctx.synchronize()

    for i in range(n_bytes):
        var first:  Int = (i + 1) & 15
        var second: Int = i & 15
        packed_host[i] = U8((first << 4) | second)

    for i in range(n_nf4_blocks):
        absmax_q_host[i] = U8(i & 255)

    for i in range(CODE2_SIZE):
        var x: Float64 = (Float64(i) - 127.5) / 127.5
        code2_host[i] = F32(x)

    for g in range(n_groups):
        absmax2_host[g] = F32(1.5)

    var packed_dev   = ctx.enqueue_create_buffer[packed_dtype](n_bytes)
    var absmax_q_dev = ctx.enqueue_create_buffer[packed_dtype](n_nf4_blocks)
    var code2_dev    = ctx.enqueue_create_buffer[f32_dtype](CODE2_SIZE)
    var absmax2_dev  = ctx.enqueue_create_buffer[f32_dtype](n_groups)
    var out_dev      = ctx.enqueue_create_buffer[u32_dtype](n_bytes)

    ctx.enqueue_copy(dst_buf=packed_dev,   src_buf=packed_host)
    ctx.enqueue_copy(dst_buf=absmax_q_dev, src_buf=absmax_q_host)
    ctx.enqueue_copy(dst_buf=code2_dev,    src_buf=code2_host)
    ctx.enqueue_copy(dst_buf=absmax2_dev,  src_buf=absmax2_host)
    ctx.synchronize()

    var grid_x: Int = ceildiv(n_rows, TILE_ROWS)
    var grid_y: Int = ceildiv(n_bytes_per_row, TILE_BYTES_X)

    var compiled = ctx.compile_function_checked[
        nf4_dequant_bnb_unroll2_packed_u32_f16, nf4_dequant_bnb_unroll2_packed_u32_f16
    ]()

    @always_inline
    @parameter
    fn run_kernel(bench_ctx: DeviceContext) raises:
        bench_ctx.enqueue_function_checked(
            compiled,
            packed_dev,
            absmax_q_dev,
            code2_dev,
            absmax2_dev,
            offset,
            out_dev,
            n_rows,
            n_bytes_per_row,
            grid_dim=(grid_x, grid_y),
            block_dim=(THREADS_X, THREADS_Y),
        )

    run_kernel(ctx)
    ctx.synchronize()

    var total_ns = ctx.execution_time[run_kernel](num_iters)
    return Float64(total_ns) / 1e9

fn bench_layer_unroll2_bf16(ctx: DeviceContext, n_rows: Int, n_cols: Int, num_iters: Int) raises -> Float64:
    """Benchmarks the NF4 dequantization kernel for bf16 output on a single layer.

    Measures the execution time for dequantizing a 2D weight matrix to packed u32 (bf16) over multiple iterations.

    Parameters:
        ctx: The device context for GPU operations
        n_rows: Number of rows in the matrix
        n_cols: Number of columns in the matrix
        num_iters: Number of iterations to run the benchmark

    Returns:
        Total execution time in seconds

    Note: Initializes test data with patterns to simulate real quantization states
    """
    var n_weights = n_rows * n_cols
    var n_bytes = n_weights >> 1
    var n_bytes_per_row = n_cols >> 1

    var n_nf4_blocks: Int = ceildiv(n_bytes, NF4_BYTES_PER_BLOCK)
    var n_groups: Int = ceildiv(n_nf4_blocks, STATE2_BLOCKS)
    if n_groups < 1: n_groups = 1

    var offset: F32 = 0.01

    var packed_host   = ctx.enqueue_create_host_buffer[packed_dtype](n_bytes)
    var absmax_q_host = ctx.enqueue_create_host_buffer[packed_dtype](n_nf4_blocks)
    var code2_host    = ctx.enqueue_create_host_buffer[f32_dtype](CODE2_SIZE)
    var absmax2_host  = ctx.enqueue_create_host_buffer[f32_dtype](n_groups)

    ctx.synchronize()

    for i in range(n_bytes):
        var first:  Int = (i + 1) & 15
        var second: Int = i & 15
        packed_host[i] = U8((first << 4) | second)

    for i in range(n_nf4_blocks):
        absmax_q_host[i] = U8(i & 255)

    for i in range(CODE2_SIZE):
        var x: Float64 = (Float64(i) - 127.5) / 127.5
        code2_host[i] = F32(x)

    for g in range(n_groups):
        absmax2_host[g] = F32(1.5)

    var packed_dev   = ctx.enqueue_create_buffer[packed_dtype](n_bytes)
    var absmax_q_dev = ctx.enqueue_create_buffer[packed_dtype](n_nf4_blocks)
    var code2_dev    = ctx.enqueue_create_buffer[f32_dtype](CODE2_SIZE)
    var absmax2_dev  = ctx.enqueue_create_buffer[f32_dtype](n_groups)
    var out_dev      = ctx.enqueue_create_buffer[u32_dtype](n_bytes)

    ctx.enqueue_copy(dst_buf=packed_dev,   src_buf=packed_host)
    ctx.enqueue_copy(dst_buf=absmax_q_dev, src_buf=absmax_q_host)
    ctx.enqueue_copy(dst_buf=code2_dev,    src_buf=code2_host)
    ctx.enqueue_copy(dst_buf=absmax2_dev,  src_buf=absmax2_host)
    ctx.synchronize()

    var grid_x: Int = ceildiv(n_rows, TILE_ROWS)
    var grid_y: Int = ceildiv(n_bytes_per_row, TILE_BYTES_X)

    var compiled = ctx.compile_function_checked[
        nf4_dequant_bnb_unroll2_packed_u32_bf16, nf4_dequant_bnb_unroll2_packed_u32_bf16
    ]()

    @always_inline
    @parameter
    fn run_kernel(bench_ctx: DeviceContext) raises:
        bench_ctx.enqueue_function_checked(
            compiled,
            packed_dev,
            absmax_q_dev,
            code2_dev,
            absmax2_dev,
            offset,
            out_dev,
            n_rows,
            n_bytes_per_row,
            grid_dim=(grid_x, grid_y),
            block_dim=(THREADS_X, THREADS_Y),
        )

    run_kernel(ctx)
    ctx.synchronize()

    var total_ns = ctx.execution_time[run_kernel](num_iters)
    return Float64(total_ns) / 1e9


########################
# Unsloth-like Harness #
########################
def main():
    """Main entry point for running NF4 dequantization benchmarks and correctness tests.

    This function sets up the device context, runs correctness tests for f16 and bf16,
    and performs benchmarks for mixed precision configurations similar to Unsloth tests.
    """
    with DeviceContext() as ctx:
        print("=" * 43)
        print("NF4 Dequant (bnb-correct) | Mixed Precision")
        print("=" * 43)
        print("Tile:", TILE_BYTES_X, "bytes/row  | Rows/tile:", TILE_ROWS)
        print("Block:", THREADS_X, "x", THREADS_Y, "=", THREADS_X * THREADS_Y, "threads")
        print("Each thread: 2 bytes via unroll2")
        print("2 u32 stores (packed 2x16-bit floats)")
        print("Device:", ctx.api())

        print("\n--- Correctness (F16) ---")
        run_correctness_test_f16(ctx)

        print("\n--- Correctness (BF16) ---")
        run_correctness_test_bf16(ctx)

        var NUM_ITERS: Int = 1000

        # Unsloth test_dequantize options:
        # (bsz, qlen, hd, m, seed, dtype)
        # (2, 3333, 2048,  8192, 3407, fp16)
        # (5,  777, 1024,  4096, 3409, bf16)
        # (3, 2048, 4096, 14336, 3408, bf16)
        var hd1: Int = 2048
        var m1:  Int = 8192
        var hd2: Int = 1024
        var m2:  Int = 4096
        var hd3: Int = 4096
        var m3:  Int = 14336

        # MLP weight shapes (bnb Linear4bit):
        # up_proj   : (m,  hd)
        # gate_proj : (m,  hd)
        # down_proj : (hd, m)

        @always_inline
        fn bench_config_f16(ctx: DeviceContext, hd: Int, m: Int, iters: Int) raises -> Float64:
            var t_up   = bench_layer_unroll2_f16(ctx, m,  hd, iters)
            var t_gate = bench_layer_unroll2_f16(ctx, m,  hd, iters)
            var t_down = bench_layer_unroll2_f16(ctx, hd, m,  iters)
            return t_up + t_gate + t_down

        @always_inline
        fn bench_config_bf16(ctx: DeviceContext, hd: Int, m: Int, iters: Int) raises -> Float64:
            var t_up   = bench_layer_unroll2_bf16(ctx, m,  hd, iters)
            var t_gate = bench_layer_unroll2_bf16(ctx, m,  hd, iters)
            var t_down = bench_layer_unroll2_bf16(ctx, hd, m,  iters)
            return t_up + t_gate + t_down

        #########################
        # Mixed FP16 x1 BF16 x2 #
        #########################
        print("\n" + "-" * 35)
        print("MIXED - FP16 x1, BF16 x2")
        print("-" * 35)

        var total_mixed: Float64 = 0.0

        print("Config 1 [FP16]: hd=", hd1, " m=", m1)
        var c1_mixed = bench_config_f16(ctx, hd1, m1, NUM_ITERS)
        print("  total:", c1_mixed, "s")
        total_mixed += c1_mixed

        print("Config 2 [BF16]: hd=", hd2, " m=", m2)
        var c2_mixed = bench_config_bf16(ctx, hd2, m2, NUM_ITERS)
        print("  total:", c2_mixed, "s")
        total_mixed += c2_mixed

        print("Config 3 [BF16]: hd=", hd3, " m=", m3)
        var c3_mixed = bench_config_bf16(ctx, hd3, m3, NUM_ITERS)
        print("  total:", c3_mixed, "s\n")
        total_mixed += c3_mixed

        print("MIXED TOTAL:", total_mixed, "s")

        print("\n" + "=" * 35)
        print("  MIXED:", total_mixed, "s")
        print("=" * 35)

NF4 Dequant (bnb-correct) | Mixed Precision
Tile: 256 bytes/row  | Rows/tile: 4
Block: 128 x 4 = 512 threads
Each thread: 2 bytes via unroll2
2 u32 stores (packed 2x16-bit floats)
Device: cuda

--- Correctness (F16) ---
[F16] max_abs_diff = 0.0
[F16] Correctness: PASSED

--- Correctness (BF16) ---
[BF16] max_abs_diff = 0.0
[BF16] Correctness: PASSED

-----------------------------------
MIXED - FP16 x1, BF16 x2
-----------------------------------
Config 1 [FP16]: hd= 2048  m= 8192
  total: 0.7430141889999999 s
Config 2 [BF16]: hd= 1024  m= 4096
  total: 0.172033949 s
Config 3 [BF16]: hd= 4096  m= 14336
  total: 2.484810362 s

MIXED TOTAL: 3.3998585 s

  MIXED: 3.3998585 s



## Calculate The Effective GB/s

In [12]:
iters = 1000
layers = 3
configs = [
    (2048, 8192),
    (1024, 4096),
    (4096, 14336),
]
total_weights = layers * iters * sum(r*c for r, c in configs)

# Traffic model (bytes moved per dequantized FP16 value)
# - output fp16: 2 bytes
# - packed nf4: 0.5 bytes (1 byte -> 2 weights)
# - absmax_q: 1 byte per 64 weights
# - absmax2: 4 bytes per (64 weights * 256 blocks) = 16384 weights
bytes_per_weight = 2.0 + 0.5 + (1/64) + (4/(64*256))

def effective_gbps(seconds: float, bytes_per_weight: float = bytes_per_weight) -> float:
    return (total_weights * bytes_per_weight) / (seconds * 1e9)

times_unsloth = {
    "T4": 3.6984355449676514,
    "\nL4": 3.0031237602233887,
    "\nA100 40GB": 1.2074337005615234,
    "\nH100 PCIe": 0.6175928115844727
    }
times_mojo    = {
    "T4": 3.461897315,
    "\nL4": 2.404278076,
    "\nA100 40GB": 0.656471045,
    "\nH100 PCIe": 0.408032219
    }

for gpu in ["T4","\nL4","\nA100 40GB","\nH100 PCIe"]:
    print(gpu, "\nunsloth GB/s", effective_gbps(times_unsloth[gpu]),
               "\nmojo    GB/s",    effective_gbps(times_mojo[gpu]))


T4 
unsloth GB/s 162.63153235654426 
mojo    GB/s 173.7435242211972

L4 
unsloth GB/s 200.28553200726515 
mojo    GB/s 250.1716610920009

A100 40GB 
unsloth GB/s 498.1492894560401 
mojo    GB/s 916.2357495904484

H100 PCIe 
unsloth GB/s 973.9139263244662 
mojo    GB/s 1474.104769162849
